In [1]:
using BenchmarkTools

In [2]:
GP = 7
C1 = [rand(3) for  i=1:5];
C2 = [rand(3) for  i=1:5];

In [3]:
function Gauss(n, eps=1e-4)
    A = zeros(n)  # Preallocations.
    W = zeros(n)
    m = Int((n+1)/2)
    pp=0
    for i=1:m
        z = cos(pi*(i - 0.25)/(n + 0.5)) # Initial estimate.
        z1 = z + 1

        while abs(z-z1)>eps
            p1 = 1
            p2 = 0

            for j=1:n
                p3 = p2
                p2 = p1
                p1 = ((2*j-1)*z*p2-(j-1)*p3)/j # The Legendre polynomial.
            end
            pp = n*(z*p1-p2)/(z^2-1)   # The L.P. derivative.
            z1 = z
            z = z1-p1/pp
        end
        A[i] = -z      # Build up the abscissas.
        A[n+1-i] = z
        W[i] = 2/((1-z^2)*(pp^2))  # Build up the weights.
        W[n+1-i] = W[i]
    end
    return A, W
end

Gauss (generic function with 2 methods)

In [4]:
function F(s::Float64,
        t::Array{Float64, 1},
        P1::Array{Float64, 1},
        P2::Array{Float64, 1},
        P3::Array{Float64, 1},
        P4::Array{Float64, 1})::Array{Float64, 1}
    
    T = (t .+ 1.0)/2.0
    X1 = P1' .+ T * (P2 - P1)'
    S = (s .+ 1.0)/2.0
    X2 = P3 .+ S .* (P4 - P3)
    R = dropdims(sum((X1' .- X2).^2, dims=1),dims=1)
    return 0.5.*log.(R) .* sum((P2-P1).*(P4-P3))
end                                                      

F (generic function with 1 method)

In [5]:
A, W = Gauss(GP);

In [26]:
function VF(C1::Array{Array{Float64, 1},1}, 
            C2::Array{Array{Float64, 1},1}, 
            A::Array{Float64, 1}, 
            W::Array{Float64, 1})::Float64
    L1 = length(C1)-1;
    L2 = length(C2)-1;
    GP = length(A)
    S = 0.0
    for i=1:L1 # Loop over segment pairs.
        P1 = C1[i]
        P2 = C1[i+1]
        for j=1:L2      
            SM = 0.0
            P3 = C2[j]
            P4 = C2[j+1]
            # Next perform the Gauss-Legendre quadrature
            for k=1:GP
                SM += sum(W[k] .* W .* F(A[k],A,P1,P2,P3,P4))
            end
            S += SM
        end
    end
    return S
end

VF (generic function with 1 method)

In [27]:
@btime VF(C1, C2, A, W)

  112.592 μs (2352 allocations: 255.50 KiB)


-2.8239450019555092

In [8]:
P1,P2,P3,P4 = C1[1], C1[2], C2[1], C2[2]

([0.443297, 0.243971, 0.620686], [0.135101, 0.446645, 0.182843], [0.688806, 0.565522, 0.0318561], [0.506535, 0.791719, 0.833558])

In [9]:
A = (A .+ 1.0)/2.0
X1 = [P1 .+ a * (P2 - P1) for a in A]

7-element Array{Array{Float64,1},1}:
 [0.435454, 0.249128, 0.609544]
 [0.403467, 0.270163, 0.564101]
 [0.351739, 0.304181, 0.490612]
 [0.289199, 0.345308, 0.401764]
 [0.226659, 0.386435, 0.312916]
 [0.174931, 0.420452, 0.239427]
 [0.142943, 0.441487, 0.193984]

In [10]:
X2 = [P3 .+ a .* (P4 - P3) for a in A]

7-element Array{Array{Float64,1},1}:
 [0.684168, 0.571278, 0.0522562]
 [0.66525, 0.594754, 0.135464]  
 [0.634657, 0.63272, 0.270024]  
 [0.59767, 0.678621, 0.432707]  
 [0.560684, 0.724521, 0.595391] 
 [0.530091, 0.762487, 0.729951] 
 [0.511173, 0.785964, 0.813158] 

In [11]:
@btime [sum(r) for r in  [(x1 .- x2).^2 for x1 in X1, x2 in X2]]

  13.037 μs (258 allocations: 12.75 KiB)


7×7 Array{Float64,2}:
 0.476209  0.397016  0.302099  0.24205   0.241882  0.286991  0.335385
 0.431448  0.35762   0.271378  0.221817  0.232137  0.285922  0.33968 
 0.374006  0.308853  0.236641  0.204041  0.231323  0.299137  0.36157 
 0.329219  0.274554  0.219303  0.20721   0.254998  0.339774  0.412696
 0.311425  0.267249  0.228959  0.237372  0.305667  0.407404  0.490814
 0.317104  0.281603  0.257343  0.282717  0.367974  0.48374   0.575825
 0.329856  0.29972   0.284135  0.319998  0.415742  0.540184  0.637634

In [28]:
function F2(A::Array{Float64, 1},
        P1::Array{Float64, 1},
        P2::Array{Float64, 1},
        P3::Array{Float64, 1},
        P4::Array{Float64, 1})::Array{Float64, 2}
    
    A = (A .+ 1.0)/2.0
    X1 = [P1 .+ a .* (P2-P1) for a in A]
    X2 = [P3 .+ a .* (P4-P3) for a in A]
    R = [(x1 - x2).^2 for x1 in X1, x2 in X2]
    return 0.5.*log.([sum(r) for r in R]).* sum((P2 - P1).*(P4 - P3))
end      

F2 (generic function with 1 method)

In [29]:
@btime F2(A, P1, P2, P3, P4)

  14.617 μs (347 allocations: 22.67 KiB)


7×7 Array{Float64,2}:
 0.194523  0.189969  0.178532  0.15968    0.137553   0.118131   0.105976 
 0.191064  0.185886  0.173708  0.154389   0.132162   0.112834   0.100786 
 0.183616  0.177604  0.164527  0.144786   0.122656   0.103644   0.0918501
 0.172073  0.165344  0.151639  0.131852   0.110183   0.0917544  0.0803659
 0.158473  0.151318  0.137401  0.11795    0.0970014  0.0793035  0.0683884
 0.146177  0.138851  0.124998  0.106023   0.0857923  0.068762   0.0582663
 0.138264  0.130898  0.117165  0.0985452  0.0787915  0.0621888  0.0519581

In [20]:
@btime vcat([F(A[k], A, P1, P2, P3, P4)' for k=1:7]...)'

  8.033 μs (167 allocations: 16.09 KiB)


7×7 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 0.194523  0.189969  0.178532  0.15968    0.137553   0.118131   0.105976 
 0.191064  0.185886  0.173708  0.154389   0.132162   0.112834   0.100786 
 0.183616  0.177604  0.164527  0.144786   0.122656   0.103644   0.0918501
 0.172073  0.165344  0.151639  0.131852   0.110183   0.0917544  0.0803659
 0.158473  0.151318  0.137401  0.11795    0.0970014  0.0793035  0.0683884
 0.146177  0.138851  0.124998  0.106023   0.0857923  0.068762   0.0582663
 0.138264  0.130898  0.117165  0.0985452  0.0787915  0.0621888  0.0519581

In [21]:
function VF2(C1::Array{Array{Float64, 1},1}, 
            C2::Array{Array{Float64, 1},1}, 
            A::Array{Float64, 1}, 
            W::Array{Float64, 1})::Float64
    L1 = length(C1) - 1;
    L2 = length(C2) - 1;
    GP = length(A)
    S = 0.0
    for i=1:L1 # Loop over segment pairs.
        P1 = C1[i]
        P2 = C1[i+1]
        for j=1:L2      
            P3 = C2[j]
            P4 = C2[j+1]
            # Next perform the Gauss-Legendre quadrature
            S += sum(W.*F2(A, P1, P2, P3, P4).*W')
        end
    end
    return S
end

VF2 (generic function with 1 method)

In [22]:
@btime VF2(C1, C2, A, W)

  248.098 μs (5680 allocations: 382.75 KiB)


-2.8239450019555083

In [23]:
@btime VF(C1, C2, A, W)

  112.197 μs (2352 allocations: 255.50 KiB)


-2.8239450019555092

## 2d version

In [32]:
X1 = P1' .+ A * (P2-P1)'
X2 = P3' .+ A .* (P4-P3)'

7×3 Array{Float64,2}:
 0.684168  0.571278  0.0522562
 0.66525   0.594754  0.135464 
 0.634657  0.63272   0.270024 
 0.59767   0.678621  0.432707 
 0.560684  0.724521  0.595391 
 0.530091  0.762487  0.729951 
 0.511173  0.785964  0.813158 

In [46]:
repeat(X1,7)

49×3 Array{Float64,2}:
 0.435454  0.249128  0.609544
 0.403467  0.270163  0.564101
 0.351739  0.304181  0.490612
 0.289199  0.345308  0.401764
 0.226659  0.386435  0.312916
 0.174931  0.420452  0.239427
 0.142943  0.441487  0.193984
 0.435454  0.249128  0.609544
 0.403467  0.270163  0.564101
 0.351739  0.304181  0.490612
 0.289199  0.345308  0.401764
 0.226659  0.386435  0.312916
 0.174931  0.420452  0.239427
 ⋮                           
 0.351739  0.304181  0.490612
 0.289199  0.345308  0.401764
 0.226659  0.386435  0.312916
 0.174931  0.420452  0.239427
 0.142943  0.441487  0.193984
 0.435454  0.249128  0.609544
 0.403467  0.270163  0.564101
 0.351739  0.304181  0.490612
 0.289199  0.345308  0.401764
 0.226659  0.386435  0.312916
 0.174931  0.420452  0.239427
 0.142943  0.441487  0.193984

In [47]:
repeat(X2', 7)

21×7 Array{Float64,2}:
 0.684168   0.66525   0.634657  0.59767   0.560684  0.530091  0.511173
 0.571278   0.594754  0.63272   0.678621  0.724521  0.762487  0.785964
 0.0522562  0.135464  0.270024  0.432707  0.595391  0.729951  0.813158
 0.684168   0.66525   0.634657  0.59767   0.560684  0.530091  0.511173
 0.571278   0.594754  0.63272   0.678621  0.724521  0.762487  0.785964
 0.0522562  0.135464  0.270024  0.432707  0.595391  0.729951  0.813158
 0.684168   0.66525   0.634657  0.59767   0.560684  0.530091  0.511173
 0.571278   0.594754  0.63272   0.678621  0.724521  0.762487  0.785964
 0.0522562  0.135464  0.270024  0.432707  0.595391  0.729951  0.813158
 0.684168   0.66525   0.634657  0.59767   0.560684  0.530091  0.511173
 0.571278   0.594754  0.63272   0.678621  0.724521  0.762487  0.785964
 0.0522562  0.135464  0.270024  0.432707  0.595391  0.729951  0.813158
 0.684168   0.66525   0.634657  0.59767   0.560684  0.530091  0.511173
 0.571278   0.594754  0.63272   0.678621  0.724521  0.